In [4]:
import pandas as pd
import numpy as np
import scipy as sp
import matplotlib.pyplot as plt
import sklearn as sk 

In [5]:
from sklearn.datasets import load_breast_cancer

In [6]:
df = load_breast_cancer()

In [16]:
x, Y = df.data, df.target

In [17]:
from sklearn.preprocessing import LabelEncoder
le = LabelEncoder()
y = le.fit_transform(Y)

In [20]:
import warnings
warnings.filterwarnings("ignore")

In [21]:
from sklearn.cross_validation import train_test_split
x_train, y_train, x_test, y_test = train_test_split(x, y , test_size= 0.20, random_state = 1)

# combining transformers and estimators in a pipeline

In [24]:
from sklearn.preprocessing import StandardScaler
from sklearn.decomposition import PCA
from sklearn.linear_model import LogisticRegression
from sklearn.pipeline import Pipeline

pipe_lr = Pipeline([('scl', StandardScaler()), ('pca', PCA(n_components = 2)), 
                   ('clf', LogisticRegression(random_state = 1))])

pipe_lr.fit(x_train,y_train)
print ('Test accuracy: %.3f' % pipe_lr.score(x_test, y_test))

ValueError: bad input shape (114, 30)

# Using k-fold cross-validation to assess model performance

In [34]:
y.shape

(569,)

In [38]:
from sklearn.cross_validation import StratifiedKFold
kfold = StratifiedKFold(y = y_train, n_folds = 6, random_state = 1)
scores = []

for k, (train, test) in enumerate(kfold):
    pip_lr.fit(x_train[train], y_train[train])
    score = pipe_lr.score(x_train[test], y_train[test])
    scores.append(score)
    print ("Fold: %S , Class dist.: %S, Acc: %.3f" % (k+1, np.bincount(y_train[train]), score))

IndexError: too many indices for array

In [39]:
print ('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

NameError: name 'scores' is not defined

In [43]:
from sklearn.cross_validation import cross_val_score
scores = cross_val_score(estimator = pipe_lr,X = x_train, cv =10, n_jobs = 1)

print ('CV accuracy scores: %s'  % scores)

ValueError: Expected array-like (array or non-string sequence), got None

# diagnosing bias and variance problems with learning curves

In [46]:
# lets see how we can use the learning curve function from scikit-learn to evaluate the model

from sklearn.learning_curve import learning_curve
pipe_lr = Pipeline([('scl', StandardScaler()), ('clf',LogisticRegression(penalty = 'l2', random_state = 0))])

train_sizes, train_scores, test_scores = learning_curve(estimator = pipe_lr, 
                                                       X  = x_train, y = y_train,
                                                       train_sizes = np.linspace(0.1,1.0,10), 
                                                       cv = 10, n_jobs = 1)

train_mean = np.mean(train_scores, axis = 1)
train_std = np.std(train_scores, axis = 1)
test_mean = np.mean(test_scores, axis = 1)
test_std = np.std(test_scores, axis = 1)
plt.plot(train_sizes, train_mean, color = 'blue', marker = 'o', markersize =5, 
        label = 'training accuracy')
plt.fill_between(train_sizes, train_mean + train_std, train_mean-train_std,alpha = 0.15, color = 'blue')
plt.plot(train_sizes, test_mean, color = 'green', marker = 's', markersize = 5,
        label = 'validation accuracy')
plt.fill_between(train_sizes, test_mean+ test_std, test_mean-test_std, alpha = 0.15, color = 'green')
plt.grid()
plt.xlabel('Number of training samples')
plt.ylabel('Accuracy')
plt.legend(loc = 'lower right')
plt.ylim([0.8, 1.0])
plt.show()

ValueError: Found input variables with inconsistent numbers of samples: [455, 114]

# Addressing ovefitting and underfitting with validation curves

In [65]:
X_train = x_train[:114, :]
#X_train.shape
from sklearn.learning_curve import validation_curve
param_range = [0.001, 0.01, 0.1, 1.0, 10.0, 100.0]
train_scores, test_scores = validation_curve(estimator = pipe_lr, X= X_train, y = y_train, 
                                            param_name = 'C', param_range = param_range, 
                                            cv =10)
train_mean = np.mean(train_scores, axis = 1)
train_std = np.std(train_scores, axis = 1)
test_mean = np.mean(test_scores, axis = 1)
test_std = np.std(test_scores, axis = 1)

ValueError: Invalid parameter C for estimator Pipeline. Check the list of available parameters with `estimator.get_params().keys()`.

In [50]:
plt.plot(param_range, train_mean, color = 'blue', marker = 'o', markersize = 5, 
        label = 'training accuracy')

plt.fill_between(param_range, train_mean+ train_std, train_mean - train_std, alpha = 0.15, color = 'blue')

plt.plot(param_range, test_mean, color = 'green', linestyle = '--', marker = 's',
        markersize = 5,label = 'validation accuracy')
plt.fill_between(param_range, test_mean + test_std, test_mean - test_std, alpha = 0.15, color = 'green')
plt.grid()
plt.xscale('log')
plt.legend(loc = 'lower right')
plt.xlabel('Parameter C')
plt.ylabel('Accuracy')
plt.ylim([0.8,1.0])
plt.show()

NameError: name 'train_mean' is not defined

# fine-tuning machine learning models via grid search

In [51]:
from sklearn.grid_search import GridSearchCV

In [55]:
X_train = x_train[:114, :]
X_train.shape

(114, 30)

In [70]:
from sklearn.svm import SVC
pipe_svc = Pipeline([('scl', StandardScaler()),('clf',SVC(random_state = 1))])
param_range = [0.0001,0.001,0.01,0.1,1.0,10.0,100.0,1000.0]
param_grid = [{'C':param_range, 'kernel':['linear']},{'C':param_range,'gamma': param_range,
                                                             'kernel':['rbf']}]
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, scoring = 'accuracy',
                 cv = 10, n_jobs= -1)

gs = gs.fit(X_train, y_train)
print(gs.best_score_)

print (gs.best_param_)

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\ACER\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\ACER\Anaconda3\lib\runpy.py i

In [60]:
from sklearn.linear_model import LinearRegression
lr = LinearRegression()
lr.get_params().keys()

dict_keys(['copy_X', 'fit_intercept', 'n_jobs', 'normalize'])

In [61]:
from sklearn.linear_model import LogisticRegression

In [62]:
logre = LogisticRegression()
logre.get_params().keys()

dict_keys(['C', 'class_weight', 'dual', 'fit_intercept', 'intercept_scaling', 'max_iter', 'multi_class', 'n_jobs', 'penalty', 'random_state', 'solver', 'tol', 'verbose', 'warm_start'])

In [67]:
from sklearn.svm import SVC
SC =SVC()
SC.get_params().keys()

dict_keys(['C', 'cache_size', 'class_weight', 'coef0', 'decision_function_shape', 'degree', 'gamma', 'kernel', 'max_iter', 'probability', 'random_state', 'shrinking', 'tol', 'verbose'])

# nested cross-validation

In [72]:
gs = GridSearchCV(estimator = pipe_svc, param_grid = param_grid, scoring = 'accuracy', cv = 10,
                 n_jobs = -1)
scores = cross_val_score(gs, X_train, y_train, scoring = 'accuracy', cv = 5)
print ('CV accuracy: %.3f +/- %.3f' % (np.mean(scores), np.std(scores)))

JoblibValueError: JoblibValueError
___________________________________________________________________________
Multiprocessing exception:
...........................................................................
C:\Users\ACER\Anaconda3\lib\runpy.py in _run_module_as_main(mod_name='ipykernel_launcher', alter_argv=1)
    188         sys.exit(msg)
    189     main_globals = sys.modules["__main__"].__dict__
    190     if alter_argv:
    191         sys.argv[0] = mod_spec.origin
    192     return _run_code(code, main_globals, None,
--> 193                      "__main__", mod_spec)
        mod_spec = ModuleSpec(name='ipykernel_launcher', loader=<_f...nda3\\lib\\site-packages\\ipykernel_launcher.py')
    194 
    195 def run_module(mod_name, init_globals=None,
    196                run_name=None, alter_sys=False):
    197     """Execute a module's code without importing it

...........................................................................
C:\Users\ACER\Anaconda3\lib\runpy.py i

In [73]:
from sklearn.tree import DecisionTreeClassifier
gs = GridSearchCV(estimator = DecisionTreeClassifier(random_state = 0), 
                 param_grid = [{'max_depth': [1,2,3,4,5,6,7,None]}], scoring = 'accuracy', 
                 cv = 5)

scores =cross_val_score(gs, X_train, y_train, scoring = 'accuracy', cv = 5)
print ("CV accuracy: %.3f +/- %.3f" % (np.mean(scores), np.std(scores)))

ValueError: Unknown label type: 'continuous-multioutput'

# confusion matrix

In [74]:
from sklearn.metrics import confusion_matrix
pipe_svc.fit(X_train, y_train)
y_pred = pipe_svc.predict(X_test)
confmat = confusion_matrix(y_true = y_test, y_pred = y_pred)
print (confmat)

ValueError: bad input shape (114, 30)